<a href="https://colab.research.google.com/github/dwjang0902/ExampleNew/blob/master/Read_Stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참조 사이트 :
https://bigdata-sk.tistory.com/10
Naver KOSPI200 지수 수집 크롤링 (파이썬을 활용한 금융공학 레시피 7장)
(https://github.com/sukyungheo/download/blob/master/07.1.%20%EA%B3%BC%EA%B1%B0%EC%8B%9C%EC%84%B8%20%EC%88%98%EC%A7%91.ipynb)

In [0]:
index_cd = 'KPI200'
page_n = 1
naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
print(naver_index)
from urllib.request import urlopen  
source = urlopen(naver_index).read()
#print(source)

import bs4
source = bs4.BeautifulSoup(source, 'lxml')
#print(source.prettify())

td = source.find_all('td')
print(len(td))

#날짜 추출
# /html/body/div/table[1]/tbody/tr[3]/td[1]
a = source.find_all('table')[0].find_all('tr')[2].find_all('td')[0]
print(a)
d = source.find_all('td', class_='date')[0].text
print(d)

import datetime as dt
yyyy = int(d.split('.')[0]) 
mm = int(d.split('.')[1])
dd = int(d.split('.')[2])
this_date= dt.date(yyyy, mm, dd)
print(this_date)

def date_format(d): #함수로 정의
    d = str(d).replace('-', '.')
    yyyy = int(d.split('.')[0]) 
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    this_date= dt.date(yyyy, mm, dd)
    return this_date

#종가추출
# /html/body/div/table[1]/tbody/tr[3]/td[2]  
this_close = source.find_all('tr')[2].find_all('td')[1].text
this_close = this_close.replace(',', '')
this_close = float(this_close)
this_close
print(this_close) 

p = source.find_all('td', class_='number_1')[0].text
print(p)

#페이지 상의 날짜와 종가정보 전체 추출
dates = source.find_all('td', class_='date')
prices = source.find_all('td', class_='number_1')
print(len(dates)) 
print(len(prices))

for n in range(len(dates)):
    this_date = dates[n].text
    this_date = date_format(this_date)
    
    this_close = prices[n*4].text   
    # 0, 4, 8, ... 4의 배수로 돌아가는 가격 추출
    this_close = this_close.replace(',', '')
    this_close = float(this_close)
    this_close
    print(this_date, this_close)
    
#마지막 페이지 번호 찾기
# /html/body/div/table[2]/tbody/tr/td[7]/a
paging = source.find('td', class_='pgRR').find('a')['href']
print(paging)

paging = paging.split('&')[1]
print(paging) 
paging = paging.split('=')[1]
print(paging)    

naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(505)
print(naver_index)
source = urlopen(naver_index).read()
source = bs4.BeautifulSoup(source, 'lxml')

if source.find('td', class_='pgRR'):
    last_page = source.find('td', class_='pgRR').find('a')['href']
    last_page = last_page.split('&')[1]
    last_page = last_page.split('=')[1]
    last_page = int(last_page)
    
def historical_index_naver(index_cd, page_n=1, last_page=0):   
        
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            # 종가 처리
            this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
            this_close = this_close.replace(',', '')
            this_close = float(this_close)

            # 딕셔너리에 저장
            historical_prices[this_date] = this_close
            
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices


#네이버에서 일자별 인덱스를 추출하는 함수 정의
    
def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:   
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
        
        
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices              
            
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices

index_cd = 'KPI200'
historical_prices = dict()
historical_index_naver(index_cd, '2018-4-1', '2018-4-4')
historical_prices


#다음에서 해외지수 추출
url = 'http://finance.daum.net/global/index_daily.daum?type=default&ric=/.GSPC&page=1' 
    
source = urlopen(url).read()
source = bs4.BeautifulSoup(source, 'lxml')

dates = source.find_all('td', class_='datetime')
dates

prices = source.find_all('td', class_='num')
print(len(dates))
print(len(prices))
#print(prices[0].text)

def historical_global_daum(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:  
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
    
    url = 'http://finance.daum.net/global/index_daily.daum?type=default&ric=/.' + index_cd + '&page=' + str(page_n)

    source = urlopen(url).read()
    source = bs4.BeautifulSoup(source, 'lxml')

    dates = source.find_all('td', class_='datetime')   # <td class="datetime">태그에서 날짜 수집
    prices = source.find_all('td', class_='num')   # <td class="num">태그에서 날짜 수집

    rows_in_page = len(dates)

    if len(dates) > 0:

        for n in range(rows_in_page):

            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)

            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*3].text
                this_close = this_close.replace(' ', '')
                this_close = this_close.replace('\t', '')
                this_close = this_close.replace('\n', '')
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices                         
        # 페이지 네비게이션
        if rows_in_page == 10:
            page_n = int(page_n)
            page_n = page_n + 1
            
            historical_global_daum(index_cd, start_date, end_date, page_n, last_page)
            
    return historical_prices


historical_prices = dict()
daum = historical_global_daum('GSPC', '2018-4-1', '2018-4-5')
print(daum)

#만들어둔 함수를 이용해 KOSPI200과 S&P500 지수 추출  
index_cd = 'KPI200'
historical_prices = dict()
kospi200 = historical_index_naver(index_cd, '2017-1-1', '2017-1-31')

index_cd = 'GSPC'
historical_prices = dict()
sp500 = historical_global_daum(index_cd, '2017-1-1', '2017-1-31')
tmp = {'S&P500':sp500, 'KOSPI200':kospi200}


import pandas as pd

df = pd.DataFrame(tmp)
print(df)

df = df.fillna(method='ffill')
if df.isnull().values.any():
    df = df.fillna(method='bfill')
print(df)

print(df.head())

index_cd = 'KPI200'
historical_prices = dict()
kospi200 = historical_index_naver(index_cd, '2018-1-1', '2018-1-31')

index_cd = 'GSPC'
historical_prices = dict()
sp500 = historical_global_daum(index_cd, '2018-1-1', '2018-1-31')

tmp = {'S&P500':sp500, 'KOSPI200':kospi200}

df = pd.DataFrame(tmp)
print(df)

df = df.fillna(method='ffill')
if df.isnull().values.any():
    df = df.fillna(method='bfill')
print(df)

"""
#matplotlib를 이용해 그래프 그리기
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 5))
plt.plot(df['S&P500'])
plt.plot(df['KOSPI200'])
plt.legend(loc=0)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)

df.iloc[0]

df.loc[dt.date(2008, 1, 2)]

plt.figure(figsize=(10, 5))
plt.plot(df['S&P500'] / df['S&P500'].loc[dt.date(2008, 1, 2)] * 100)
plt.plot(df['KOSPI200'] / df['KOSPI200'].loc[dt.date(2008, 1, 2)] * 100)
plt.legend(loc=0)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)

df_ratio_2016_now = df.loc[dt.date(2016, 1, 1):] / df.loc[dt.date(2016, 1, 4)] * 100
df_ratio_2016_now.head(3)

plt.figure(figsize=(10, 5))
plt.plot(df_ratio_2016_now['S&P500'])
plt.plot(df_ratio_2016_now['KOSPI200'])
plt.legend(loc=0)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)

plt.figure(figsize=(7,7))
plt.scatter(df_ratio_2016_now['S&P500'], df_ratio_2016_now['KOSPI200'], marker='.')
plt.grid(True, color='0.7', linestyle=':', linewidth=1)
plt.xlabel('S&P500')
plt.ylabel('KOSPI200')

import numpy as np
from sklearn.linear_model import LinearRegression

x = df_ratio_2016_now['S&P500']
y = df_ratio_2016_now['KOSPI200']

# 1개 컬럼 np.array로 변환
independent_var = np.array(x).reshape(-1, 1)
dependent_var = np.array(y).reshape(-1, 1)

# Linear Regression
regr = LinearRegression()
regr.fit(independent_var, dependent_var)

result = {'Slope':regr.coef_[0,0], 'Intercept':regr.intercept_[0], 'R^2':regr.score(independent_var, dependent_var) }
result

plt.figure(figsize=(7,7))
plt.scatter(independent_var, dependent_var, marker='.', color='skyblue')
plt.plot(independent_var, regr.predict(independent_var), color='r', linewidth=3)
plt.grid(True, color='0.7', linestyle=':', linewidth=1)
plt.xlabel('S&P500')
plt.ylabel('KOSPI200')
"""